# Redis

REmote DIctionary Service is a key-value database.

- [Official docs](https://redis.io/documentation)
- [Use cases](https://redislabs.com/solutions/use-cases/)
- More about [redis-py](https://github.com/andymccurdy/redis-py)


## Concepts

Redis is a very simple database conceptually. From a programmer perspective, it's as if you can magically persist simple values, dictionaries, sets, lists, and priority queues, so that they are usable from other programs, possibly residing in other computers. The API is simple to use. And it is an in-memory database, hence extremely fast.

More advanced concepts

- Pipelines
- Expiring values
- Publish-subscribe model

## Connect to database

In [ ]:
import redis

#### Providing access information

It is common to keep access configuration information to services such as a database or cloud platform in a local file - here we use YAML.

**Note**: This file MUST be listed in `.gitignore` - otherwise anyone with access to your repository knows your password!

In [ ]:
%%file redis_auth_config.yaml
# This would normally live on disk and not be in a notebook!

host: 'localhost'
port: 6379
password: 

In [ ]:
import yaml

with open('redis_auth_config.yaml') as f:
    auth = yaml.load(f, Loader=yaml.FullLoader)
auth

In [ ]:
r = redis.Redis(
    host = auth['host'],
    port = auth['port'],
    password = auth['password']
)

In [ ]:
r.ping()

## Clear database

In [ ]:
r.flushdb()

## Simple data types

#### Set and get a single value

In [ ]:
r.set('a', 'adenosine')

In [ ]:
r.get('a')

#### Set and get multiple values

In [ ]:
r.mset(dict(c='cytosine', t='thymidine', g='guanosine'))

In [ ]:
r.mget(list('tcga'))

#### Deletion

In [ ]:
r.delete('a')

In [ ]:
r.keys()

In [ ]:
r.delete('c', 't', 'g')

In [ ]:
r.keys()

### Transactions

Transactions are achieved by creating and executing pipeline. This is useful not just for atomicity, but also to reduce communication costs.

In [ ]:
pipe = r.pipeline()
(
pipe.set('a', 0).
    incr('a').
    incr('a').
    incr('a').
    execute()
)

In [ ]:
r.get('a')

### Expiring values

You can also find the time to expiry with `ttl` (time-to-live) and convert from volatile to permanent with `persist`

In [ ]:
import time

In [ ]:
r.setex('foo', 3, 'bar')
print('get', r.get('foo'))
time.sleep(1)
print('ttl', r.ttl('foo'))
time.sleep(1)
print('ttl', r.ttl('foo'))
time.sleep(1)
print('ttl', r.ttl('foo'))
time.sleep(1)
print('get', r.get('foo'))

#### Alternative

In [ ]:
r.set('foo', 'bar')
r.expire('foo', 3)
print(r.get('foo'))
time.sleep(3)
print(r.get('foo'))

## Complex data types

In [ ]:
import warnings

warnings.simplefilter('ignore', DeprecationWarning)

### Hash

In [ ]:
r.hmset('nuc', dict(a='adenosine', c='cytosine', t='thymidine', g='guanosine'))

In [ ]:
r.hget('nuc', 'a')

In [ ]:
r.hmget('nuc', list('ctg'))

In [ ]:
r.hkeys('nuc')

In [ ]:
r.hvals('nuc')

### List

In [ ]:
r.rpush('xs', 1, 2, 3)

In [ ]:
r.lpush('xs', 4, 5, 6)

In [ ]:
r.llen('xs')

In [ ]:
r.lrange('xs', 0, r.llen('xs'))

In [ ]:
r.lrange('xs', 0, -1)

#### Using list as a queue

In [ ]:
r.lpush('q', 1, 2, 3)

In [ ]:
while r.llen('q'):
    print(r.rpop('q'))

#### Using list as stack

In [ ]:
r.lpush('q', 1, 2, 3)

In [ ]:
while r.llen('q'):
    print(r.lpop('q'))

#### Transferring values across lists

In [ ]:
r.lpush('l1', 1,2,3)

In [ ]:
while r.llen('l1'):
    r.rpoplpush('l1', 'l2')
r.llen('l1'), r.llen('l2')

In [ ]:
for key in r.scan_iter('l2'):
    print(key)

In [ ]:
r.lpush('l1', 1,2,3)

### Sets

In [ ]:
r.sadd('s1', 1,2,3)

In [ ]:
r.sadd('s1', 2,3,4)

In [ ]:
r.smembers('s1')

In [ ]:
r.sadd('s2', 4,5,6)

In [ ]:
r.sdiff(['s1', 's2'])

In [ ]:
r.sinter(['s1', 's2'])

In [ ]:
r.sunion(['s1', 's2'])

### Sorted sets

This is equivalent to a priority queue.

In [ ]:
r.zadd('jobs', 
       dict(job1=3, 
            job2=7, 
            job3=1, 
            job4=2,
            job5=6)
      )

In [ ]:
r.zincrby('jobs', 2, 'job5')

In [ ]:
r.zrange('jobs', 0, -1, withscores=True)

In [ ]:
r.zrevrange('jobs', 0, -1, withscores=True)

#### Union and intersection store

This just creates new sets from the union and intersection respectively.

In [ ]:
s1 = 'time flies like an arrow'
s2 = 'fruit flies like a banana'

In [ ]:
from collections import Counter

In [ ]:
c1 = Counter(s1.split())

In [ ]:
c2 = Counter(s2.split())

In [ ]:
r.zadd('c1', c1)

In [ ]:
r.zadd('c2', c2)

In [ ]:
r.zrange('c1', 0, -1, withscores=True)

In [ ]:
r.zrange('c2', 0, -1, withscores=True)

In [ ]:
r.zunionstore('c3', ['c1', 'c2'])

In [ ]:
r.zrange('c3', 0, -1, withscores=True)

In [ ]:
r.zinterstore('c4', ['c1', 'c2'])

In [ ]:
r.zrange('c4', 0, -1, withscores=True)

### Publisher/Subscriber

![](https://making.pusher.com/images/2017-03-01-redis-pubsub-under-the-hood/clients.svg)

Source: https://making.pusher.com/redis-pubsub-under-the-hood/

In [ ]:
help(r.pubsub)

In [ ]:
p = r.pubsub()

#### Channels

In [ ]:
p.subscribe('python', 'perl', 'sql')

In [ ]:
m = p.get_message()
while m:
    print(m)
    m = p.get_message()

In [ ]:
p.channels

In [ ]:
p2 = r.pubsub()

In [ ]:
p2.psubscribe('p*')

In [ ]:
p2.patterns

#### Messages

From [redis-puy](https://github.com/andymccurdy/redis-py)

Every message read from a PubSub instance will be a dictionary with the following keys.

- type: One of the following: 'subscribe', 'unsubscribe', 'psubscribe', 'punsubscribe', 'message', 'pmessage'
- channel: The channel [un]subscribed to or the channel a message was published to
- pattern: The pattern that matched a published message's channel. Will be None in all cases except for 'pmessage' types.
- data: The message data. With [un]subscribe messages, this value will be the number of channels and patterns the connection is currently subscribed to. With [p]message messages, this value will be the actual published message.

In [ ]:
r.publish('python', 'use blank spaces')
r.publish('python', 'no semi-colons')
r.publish('perl', 'use spaceship operator')
r.publish('sql', 'select this')
r.publish('haskell', 'functional is cool')

In [ ]:
m = p.get_message()
while m:
    print(m)
    m = p.get_message()

In [ ]:
p.unsubscribe('python')

In [ ]:
p.channels

In [ ]:
r.publish('python', 'use blank spaces 2')
r.publish('python', 'no semi-colons 2')
r.publish('perl', 'use spaceship operator 2')
r.publish('sql', 'select this 2')
r.publish('haskell', 'functional is cool 2')

In [ ]:
m = p.get_message()
while m:
    print(m)
    m = p.get_message()

In [ ]:
m = p2.get_message()
while m:
    print(m)
    m = p2.get_message()

### Multiple databases

In [ ]:
r2 = redis.Redis(db=1)
r2.flushdb()

In [ ]:
for c in ['c1', 'c2', 'c3', 'c4']:
    r.move(c, 1)

In [ ]:
for key in r2.scan_iter('c?'):
    print(r2.zrange(key, 0, -1, withscores=True))

### Clean up

There is  no need to close the connections when we use the `Redis()` object. This is taken care of automatically


```python
def execute_command(self, *args, **options):
    "Execute a command and return a parsed response"
    pool = self.connection_pool
    command_name = args[0]
    connection = pool.get_connection(command_name, **options)
    try: 
        connection.send_command(*args)
        return self.parse_response(connection, command_name, **options)
    except (ConnectionError, TimeoutError) as e:
        connection.disconnect()
        if not connection.retry_on_timeout and isinstance(e, TimeoutError):
            raise
        connection.send_command(*args)
        return self.parse_response(connection, command_name, **options)
    finally:
        pool.release(connection)
 ```

#### Benchmark redis

In [ ]:
%%bash 

redis-benchmark -q -n 10000 -c 50